Now that the database has been imported, let's import the basics. I will be using Python 2.7x here, but Python 3.x should work just fine with a few tweaks.

First import the prerequisites, pandas, sqlalchemy and plotly. Install them with "pip install" if that isn't done already.

In [12]:
import pandas as pd 
from sqlalchemy import create_engine

import plotly.plotly as py
from plotly.graph_objs import *

Import the SQLite database so that it can be read with Pandas.

Note the three slashes instead of just one for a relative path.

In [13]:
disk_engine = create_engine("sqlite:///assets/turbine_locations.db")

In [14]:
df = pd.read_sql_query("Select lat_DD, long_DD, count(*) from data group by lat_DD, long_DD ", disk_engine)

Let's query the dataframe to see everything works fine so far.

In [15]:
print df

          lat_DD      long_DD  count(*)
0      18.964696  -155.690085         1
1      18.965596  -155.690085         1
2      18.966796  -155.690085         1
3      18.967996  -155.690085         1
4      18.968896  -155.690085         1
5      18.969796  -155.689985         1
6      18.970896  -155.689885         1
7      18.971996  -155.689585         1
8      18.972896  -155.689585         1
9      18.973696  -155.689585         1
10     18.974796  -155.689285         1
11     18.975896  -155.688985         1
12     18.976996  -155.688685         1
13     18.977996  -155.688385         1
14     18.989496  -155.671285         1
15     18.989696  -155.665685         1
16     18.989796  -155.671285         1
17     18.990096  -155.665785         1
18     18.990096  -155.668185         1
19     18.990196  -155.671285         1
20     18.990396  -155.665785         1
21     18.990496  -155.668185         1
22     18.990496  -155.671285         1
23     18.990796  -155.668185         1


In [16]:
print df["lat_DD"].values

[u'18.964696' u'18.965596' u'18.966796' ..., u'66.839901' u'66.839901'
 u'66.839901']


Now let's do a basic graph with Plotly. First setup your account for Plotly graphing here if you haven't used Plotly before : 

https://plot.ly/python/getting-started/

Do not skip this step as Plotly requires API authentication prior to graphing.

In [17]:
mapbox_access_token = 'pk.eyJ1IjoicGxvdGx5ZXhhbXBsZXMiLCJhIjoiY2l1azN6bGMxMDE2djJ6c2M0dmFob3pjdCJ9.iQcf9R_QqzfhgIliXixmFg'

trace = Scattermapbox(
        lat=df['lat_DD'].values,
        lon=df['long_DD'].values,
        mode='markers',
        marker=Marker(
            size=5
        ),
    )

data = Data([trace])

layout = Layout(
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=42,
            lon=-95
        ),
        pitch=0,
        zoom=5
    ),
)

fig = Figure(data=data,layout=layout)
plot_url = py.plot(fig, filename = "Mapbox 1", validate=False)

In [20]:
mapbox_access_token = 'pk.eyJ1IjoicGxvdGx5ZXhhbXBsZXMiLCJhIjoiY2l1azN6bGMxMDE2djJ6c2M0dmFob3pjdCJ9.iQcf9R_QqzfhgIliXixmFg'

trace = Scattermapbox(
        lat=df['lat_DD'].values,
        lon=df['long_DD'].values,
        mode='markers',
        marker=Marker(
            color="#FFFFFF",
            size=5
        ),
    )

data = Data([trace])

layout = Layout(
    annotations=Annotations([
        Annotation(
            x=0.00,
            y=1.07,
            showarrow=False,
            text="Map 1 : Visualization of wind turbine locations in the US",
            xref='paper',
            yref='paper'
        )
    ]),
    font=Font(family="Droid Sans", size=14, color="FFFFFF"),
    autosize=True,
    hovermode='closest',
    paper_bgcolor='rgb(40, 40, 40)',
    margin=Margin(
        t=60,
        l=20,
        b=20,
        r=20,
        pad=0
    ),
    title = "",
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=42,
            lon=-95
        ),
        pitch=0,
        zoom=5,
        style="dark"
    ),
)

fig = Figure(data=data,layout=layout)
plot_url = py.plot(fig, filename = "Mapbox 1 styled", validate=False)

Looks much better already. Now let's add color and vary the size of the points with respect to blade length.

In [23]:
df = pd.read_sql_query("Select lat_DD, long_DD, blade_l, count(*) from data group by lat_DD, long_DD, blade_l", disk_engine)

mapbox_access_token = 'pk.eyJ1IjoicGxvdGx5ZXhhbXBsZXMiLCJhIjoiY2l1azN6bGMxMDE2djJ6c2M0dmFob3pjdCJ9.iQcf9R_QqzfhgIliXixmFg'

trace = Scattermapbox(
    lat=df['lat_DD'].values,
    lon=df['long_DD'].values,
    mode='markers',
    marker=Marker(
        color=[max(float(i), 5) for i in df['blade_l'].values],
        colorscale=[[0, 'rgb(221,42,145)'], [0.35, 'rgb(177,77,236)'], [0.5, 'rgb(118,117,237)'], [0.6, 'rgb(46,142,191)'], [0.7, 'rgb(11,152,121)'], [1, 'rgb(19,152,99)']],
        reversescale=True,
        showscale=True,
        autocolorscale=False,
        colorbar=ColorBar(
            title='Turbine<br>blade<br>length (m)'
        ),
        size=[max(float(i)/5, 5) for i in df['blade_l'].values],
        opacity=0.8
      
    ),
)
 
data = Data([trace])

layout = Layout(
    annotations=Annotations([
        Annotation(
            x=0.00,
            y=1.07,
            showarrow=False,
            text="Map 2 : Visualization of wind turbine locations and their size in the US",
            xref='paper',
            yref='paper'
        )
    ]),
    font=Font(family="Droid Sans", size=14, color="FFFFFF"),
    autosize=True,
    hovermode='closest',
    paper_bgcolor='rgb(40, 40, 40)',
    margin=Margin(
        t=60,
        l=20,
        b=20,
        r=20,
        pad=0
    ),
    title = "",
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=42,
            lon=-95
        ),
        pitch=0,
        zoom=5,
        style="dark"
    ),
)

fig = Figure(data=data,layout=layout)
plot_url = py.plot(fig, filename = "Mapbox 2", validate=False)
